In [1]:
import pandas as pd
import sweetviz as sv
import numpy as np


In [2]:
df = pd.read_csv('Crime_Data_from_2020_to_Present.csv', index_col= 'DR_NO')

In [3]:
df = df.drop(
    ['Crm Cd 1', 'Crm Cd 2', 'Crm Cd 3', 'Crm Cd 4', 'LOCATION', 'Cross Street', 'Part 1-2'], 
    axis = 1
)

In [4]:
df.rename(
    columns = {
        'Date Rptd' : 'Day Reported',
        'DATE OCC' : 'Date of Occurence',
        'TIME OCC' : 'Time of Occurance',
        'Rpt Dist No' : 'Report District Number',
        'Crm Cd' : 'Crime code',
        'Crm Cd Desc' : 'Crime Description',
        'Vict Sex' : 'Victim Sex',
        'Vict Descent' : 'Victim Ethnicity',
        'Vict Age' : 'Victim Age',
        'Premis Cd' : 'Premises Code',
        'Premis Desc' : 'premises description',
        'Weapon Used Cd' : 'Weapon Used Code',
        'Weapon Desc' : 'Weapon Description',
        'Status' : 'Status Case',
        'Status Desc' : 'Status Description', 
    },
    inplace = True
)

# put on lowercase all column names
df.columns = map(str.lower, df.columns)

## Transformacion de los datos

In [5]:
df['day reported'] = df['day reported'].apply(lambda x: x[ : 10]) 
df['date of occurence'] = df['date of occurence'].apply(lambda x: x[ : 10]) 


In [6]:
# filtramos valores asi quitando irregularidades
df = df[df['time of occurance'] > 100] 
# convertimos el tipo de dato int -> str
df['time of occurance'] = df['time of occurance'].astype(str) 
# le damos formato a la fecha
df['time of occurance'] = pd.to_datetime(df['time of occurance'], format = '%H%M').dt.time

df

,day reported,date of occurence,time of occurance,area,area name,report district number,crime code,crime description,mocodes,victim age,victim sex,victim ethnicity,premises code,premises description,weapon used code,weapon description,status case,status description,lat,lon
DR_NO,,,,,,,,,,,,,,,,,,,,
190326475,03/01/2020,03/01/2020,21:30:00,7,Wilshire,784,510,VEHICLE - STOLEN,NaN,0,M,O,101.0,STREET,NaN,NaN,AA,Adult Arrest,34.0375,-118.3506
200106753,02/09/2020,02/08/2020,18:00:00,1,Central,182,330,BURGLARY FROM VEHICLE,1822 1402 0344,47,M,O,128.0,BUS STOP/LAYOVER (ALSO QUERY 124),NaN,NaN,IC,Invest Cont,34.0444,-118.2628
200320258,11/11/2020,11/04/2020,17:00:00,3,Southwest,356,480,BIKE - STOLEN,0344 1251,19,X,X,502.0,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",NaN,NaN,IC,Invest Cont,34.0210,-118.3002
200907217,05/10/2023,03/10/2020,20:37:00,9,Van Nuys,964,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),0325 1501,19,M,O,405.0,CLOTHING STORE,NaN,NaN,IC,Invest Cont,34.1576,-118.4387
220614831,08/18/2022,08/17/2020,12:00:00,6,Hollywood,666,354,THEFT OF IDENTITY,1822 1501 0930 2004,28,M,H,102.0,SIDEWALK,NaN,NaN,IC,Invest Cont,34.0944,-118.3277
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242004546,01/16/2024,01/16/2024,15:10:00,20,Olympic,2013,624,BATTERY - SIMPLE ASSAULT,2047 0400 0429 0444 1202 1822,80,F,O,124.0,BUS STOP,400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",IC,Invest Cont,34.0690,-118.3054
240710284,07/24/2024,07/23/2024,14:00:00,7,Wilshire,788,510,VEHICLE - STOLEN,NaN,0,NaN,NaN,101.0,STREET,NaN,NaN,IC,Invest Cont,34.0362,-118.3284
241711348,07/19/2024,07/19/2024,07:57:00,17,Devonshire,1751,888,TRESPASSING,1501,0,X,X,885.0,MTA - ORANGE LINE - CHATSWORTH,NaN,NaN,IC,Invest Cont,34.2500,-118.5990


In [8]:
mapping_dict_sex = {
    'F' : 'Female',
    'M' : 'Male'
}

# los valores menores a 0 los convertimos a 0 ya que son mas de 200000 filas las que presentan este caso
df.loc[df['victim age'] < 0] = 0
# borramos filas que sus unicos valores son 0
df = df.loc[~(df.eq(0).all(axis=1))] 
# Aplicamos Other si se encuentra dentro de la lista de valores pasada 
df['victim sex'] = df['victim sex'].apply(lambda x: 'Other' if x in ['X', np.nan, 'H', '-'] else x)
df['victim sex'] = df['victim sex'].replace(mapping_dict_sex)
# Para estandarizar el resto del texto usamos Title en las columnas escritas completamente con mayusculas
df[['crime description', 'premises description', 'weapon description']] = df[['crime description', 'premises description', 'weapon description']].apply(
        lambda x : x.str.title()
    )

C:\Users\horse\AppData\Local\Temp\ipykernel_19276\697385447.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['victim sex'] = df['victim sex'].apply(lambda x: 'Other' if x in ['X', np.nan, 'H', '-'] else x)
C:\Users\horse\AppData\Local\Temp\ipykernel_19276\697385447.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['victim sex'] = df['victim sex'].replace(mapping_dict_sex)
C:\Users\horse\AppData\Local\Temp\ipykernel_19276\697385447.py:14: SettingWithCopyWarning: 
A value is trying to be set on a

In [9]:
mapping_dict = {
    'O': 'Other',
    'X': 'Unknown', 
    'H': 'Hispanic', 
    'B': 'Black', 
    'W': 'White', 
    'A': 'Asian', 
    'K': 'Korean', 
    'C': 'Chinese', 
    'J': 'Japanese', 
    'F': 'Filipino', 
    'I': 'American Indian or Alaskan Native', 
    'V': 'Vietnamese', 
    'S': 'Samoan', 
    'P': 'Pacific Islander', 
    'Z': 'Asian Indian', 
    'G': 'Guamanian', 
    'U': 'Hawaiian', 
    'D': 'Cambodian', 
    'L': 'Laotian',
    np.nan : 'Unknown',
    '-' : 'Unknown'
}

df['victim ethnicity'] = df['victim ethnicity'].replace(mapping_dict)
#df['victim ethnicity'] = df['victim ethnicity'].fillna('Unknown')

C:\Users\horse\AppData\Local\Temp\ipykernel_19276\2866861391.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['victim ethnicity'] = df['victim ethnicity'].replace(mapping_dict)


In [10]:
mapping_status_dict = {
    'Adult Arrest' : 'Adult arrest',
    'Invest Cont': 'Investigation continuing',
    'Adult Other' : 'Adult other',
    'Juv Arrest' : 'Juvenile arrest',
    'Juv Other' : 'Juvenile other',
    'UNK' : 'Cleared by citation',
    np.nan : 'Unknown'
}

df['status description'] = df['status description'].replace(mapping_status_dict)

C:\Users\horse\AppData\Local\Temp\ipykernel_19276\2415216038.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['status description'] = df['status description'].replace(mapping_status_dict)


In [19]:
df['weapon used code'] = df['weapon used code'].replace(np.nan, 0)
df['weapon description'] = df['weapon description'].replace(np.nan, 'Unknown Weapon/Other Weapon')

C:\Users\horse\AppData\Local\Temp\ipykernel_19276\89119943.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['weapon used code'] = df['weapon used code'].replace(np.nan, 0)
C:\Users\horse\AppData\Local\Temp\ipykernel_19276\89119943.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['weapon description'] = df['weapon description'].replace(np.nan, 'Unknown Weapon/Other Weapon')


In [20]:
x = df['weapon description'].unique()
x

array(['Unknown Weapon/Other Weapon',
       'Strong-Arm (Hands, Fist, Feet Or Bodily Force)', 'Vehicle',
       'Verbal Threat', 'Belt Flailing Instrument/Chain', 'Hand Gun',
       'Knife With Blade 6Inches Or Less', 'Unknown Firearm',
       'Fixed Object', 'Kitchen Knife', 'Machete', 'Mace/Pepper Spray',
       'Stick', 'Other Knife', 'Physical Presence', 'Hammer',
       'Knife With Blade Over 6 Inches In Length',
       'Air Pistol/Revolver/Rifle/Bb Gun', 'Semi-Automatic Pistol',
       'Simulated Gun', 'Razor', 'Other Firearm', 'Folding Knife',
       'Pipe/Metal Pipe', 'Rock/Thrown Object', 'Rifle',
       'Other Cutting Instrument', 'Fire', 'Revolver', 'Bottle',
       'Scissors', 'Switch Blade', 'Brass Knuckles',
       'Unknown Type Cutting Instrument', 'Club/Bat', 'Blunt Instrument',
       'Board', 'Stun Gun', 'Cleaver', 'Razor Blade', 'Screwdriver',
       'Shotgun', 'Concrete Block/Brick', 'Caustic Chemical/Poison',
       'Semi-Automatic Rifle', 'Scalding Liquid', 'Tire

In [24]:
df.to_csv('Crime_Data_from_2020_to_Present_dfPreProcesado.csv')

In [22]:
# my_report = sv.analyze(df)
# my_report.show_html() # Default arguments will generate to "SWEETVIZ_REPORT.html"


                                             |          | [  0%]   00:00 -> (? left)

Report SWEETVIZ_REPORT.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.
